# Guidelines for Tuning Prompting Key Parameters
In this lesson, you'll practice how the adjustment of key paramters affect large language model responses. This will help you to write write effective prompts for large language models.

## Setup
#### Load the API key and relevant Python libaries.

For demonstation, I will use OpenAI API key

In [6]:
!pip install python-dotenv --quiet

In [14]:
import openai
import os

# Get secret API key
"""
openai_api_key = os.environ.get("OPENAI_API_KEY")
if not openai_api_key:
    raise ValueError("OpenAI API Key not found in Colab secrets!")
"""
# Set it as environment variable
os.environ["OPENAI_API_KEY"] = 'YOUR KEY'

print("API Key Loaded Successfully!")


API Key Loaded Successfully!


"\nfrom dotenv import load_dotenv, find_dotenv\n_ = load_dotenv(find_dotenv())\n\nopenai.api_key  = os.getenv('OPENAI_API_KEY')\n\nfrom openai import OpenAI\nclient = OpenAI()\n"

**helper function** <br>
<br>
Throughout this demo, we will use OpenAI's gpt-4o-mini model and the chat completions endpoint.
<br>
This helper function will make it easier to use prompts and look at the generated outputs. <br>


In [15]:
from openai import OpenAI
client = OpenAI()

def get_completion(prompt, model="gpt-4o-mini", temperature=0.5, \
                   max_tokens=40, top_p=0.9, logprobs=False):
     messages=[{"role": "user", "content": prompt}]
     response = client.chat.completions.create(
         model=model,
         messages=messages,
         temperature=temperature,
         max_tokens=max_tokens,
         top_p=top_p,
         logprobs=logprobs

     )

     return response.choices[0].message.content

### 1. Max_Tokens

In [16]:
# Define prompt
prompt = """
Write a short story about a space explorer who discovers a new planet.
"""

In [17]:
# Obtain responses: max_tokens = 5
response = get_completion(prompt, max_tokens=5)
print(response)


Captain Elara Voss


In [18]:
# Obtain responses: max_tokens = 40
response = get_completion(prompt, max_tokens=20)
print(response)

Captain Elara Voss had always been drawn to the stars. As the commander of the starship


###2. Temperature

In [28]:
# Define prompt
prompt = """
Give me a list of 3 unique and unusual ice cream flavors.
"""

In [29]:
# Obtain responses: temperature = 0
response = get_completion(prompt, temperature=0, max_tokens=100)
print(response)

Sure! Here are three unique and unusual ice cream flavors:

1. **Lavender Honey** - A delicate blend of floral lavender and sweet honey, this ice cream offers a soothing and aromatic experience, perfect for those who enjoy herbal and floral notes.

2. **Balsamic Strawberry** - This flavor combines ripe strawberries with a drizzle of aged balsamic vinegar, creating a sweet and tangy treat that enhances the natural fruitiness with a sophisticated twist.

3. **Wasabi Ginger** - A


In [30]:
response = get_completion(prompt, temperature=2, max_tokens=100)
print(response)

Sure! Here are three unique and unusual ice cream flavors:

1. **Basil and Lime Sorbet** - A refreshing sorbet that combines the herbal notes of fresh basil with the zesty brightness of lime, creating a light and invigorating dessert.

2. **Black Sesame and Coconut** - This flavor features the nutty richness of black sesame paired with the creamy sweetness of coconut, resulting in a complex and earthy ice cream that's both intriguing and delicious.

3. **Avocado and Sea Salt


### 3. Top_P (nucleus sampling)

In [31]:
# Define prompt
prompt = """
Describe a sunset over a tropical beach.
"""

In [32]:
# Obtain responses: top_p = 0.1 (conservative and repetitive)
response = get_completion(prompt, temperature=0.3, max_tokens=100, top_p=0.1)
print(response)

As the sun begins its descent toward the horizon, the sky transforms into a breathtaking canvas of colors. Hues of deep orange and vibrant pink blend seamlessly with soft purples and gentle blues, creating a stunning gradient that reflects the beauty of the tropical setting. The sun, a fiery orb, casts a warm golden light over the tranquil waters, making them shimmer like liquid gold.

The gentle waves lap rhythmically against the shore, their white foam contrasting with the rich, golden sands. Palm trees sway lightly


In [33]:
# Obtain responses: top_p = 0.9 (diverse output)
response = get_completion(prompt, temperature=0.3, max_tokens=100, top_p=0.9)
print(response)

As the sun begins its descent toward the horizon, the sky transforms into a breathtaking canvas of colors. Hues of deep orange and vibrant pink blend seamlessly with soft purples and gentle blues, casting a warm glow over the tranquil waters of the tropical beach. The sun, a fiery orb, hovers just above the horizon, its light shimmering on the surface of the waves, creating a pathway of gold that stretches toward the shore.

Palm trees sway gently in the warm evening breeze, their silhouettes dark against


### 4. log_probs

In [34]:
prompt1 = "The quick brown fox"

response1 = client.chat.completions.create(
    messages=[{"role": "user", "content": prompt1}],
    model="gpt-4o-mini",
    max_tokens=10,
    logprobs=True,
    top_logprobs = 2 # Request top 2 log probabilities for each token
)

# Print logprobs for both prompts
print("Response 1 Logprobs:")

# Access the logprobs object
logprobs = response1.choices[0].logprobs

# Iterate through the content list within logprobs
for logprobs_obj in logprobs.content:
    # Now access top_logprobs for each item in content
    if logprobs_obj.top_logprobs is not None:  # Check if top_logprobs exists
        # Access the token text using logprobs_obj.token
        print(f"Token: {logprobs_obj.token}, Top Log Probabilities: {logprobs_obj.top_logprobs}")

Response 1 Logprobs:
Token: ..., Top Log Probabilities: [TopLogprob(token='The', bytes=[84, 104, 101], logprob=-0.7013113), TopLogprob(token='"The', bytes=[34, 84, 104, 101], logprob=-0.8263113)]
Token: j, Top Log Probabilities: [TopLogprob(token='j', bytes=[106], logprob=-0.00012892624), TopLogprob(token=' jumps', bytes=[32, 106, 117, 109, 112, 115], logprob=-9.250129)]
Token: umps, Top Log Probabilities: [TopLogprob(token='umps', bytes=[117, 109, 112, 115], logprob=-1.9361265e-07), TopLogprob(token=' jumps', bytes=[32, 106, 117, 109, 112, 115], logprob=-16.625)]
Token:  over, Top Log Probabilities: [TopLogprob(token=' over', bytes=[32, 111, 118, 101, 114], logprob=0.0), TopLogprob(token=' Over', bytes=[32, 79, 118, 101, 114], logprob=-17.0)]
Token:  the, Top Log Probabilities: [TopLogprob(token=' the', bytes=[32, 116, 104, 101], logprob=0.0), TopLogprob(token=' a', bytes=[32, 97], logprob=-18.0)]
Token:  lazy, Top Log Probabilities: [TopLogprob(token=' lazy', bytes=[32, 108, 97, 122,